In [1]:
import pandas as pd
from glob import glob
import pickle
from pprint import pprint

In [2]:
data_path = "../ICLR data/masterdata_unbalanced/"

years = [2017, 2018, 2019, 2020]

final_dataset = {}

for y in years:
    papers_data = pd.read_pickle(data_path + "papers_{}.pkl".format(y))
    decisions_data = pd.read_pickle(data_path + "paper_decision_dict_{}.pkl".format(y))
    
    print("For year {}:, count of papers_data is: {} and decisions_data is {}".format(y, len(papers_data), len(decisions_data)))
    
    for k,v in decisions_data.items():
        if "decision" in decisions_data[k]:
            if "decision" in decisions_data[k]["decision"]["content"]:
                dec = decisions_data[k]["decision"]["content"]["decision"].lower()
            else:
                dec = decisions_data[k]["decision"]["content"]["recommendation"].lower()
        else:
            dec = decisions_data[k]["content"]["decision"].lower()
        
        if dec.lower().find("invite") > -1:
            continue
            print(dec)
        
        if dec.find("accept") > -1:
            dec = "Accept"
        else:
            dec = "Reject"
        
        if k in papers_data:
            if "TL;DR" not in papers_data[k]["content"]:
                papers_data[k]["content"]["TL;DR"] = None
            final_dataset[str(y) + "_" + k] = papers_data[k]
            final_dataset[str(y) + "_" + k]["label"] = dec
#         else:
#             print(decisions_data[k])
#     break

#     print(type(paper_dict))
#     print(list(paper_dict.items())[0:1])
#     break
print("\n Total papers in the dataset: ", len(final_dataset))

For year 2017:, count of papers_data is: 490 and decisions_data is 490
For year 2018:, count of papers_data is: 911 and decisions_data is 935
For year 2019:, count of papers_data is: 1419 and decisions_data is 1419
For year 2020:, count of papers_data is: 2213 and decisions_data is 2213

 Total papers in the dataset:  4897


In [3]:
paper_csv = pd.DataFrame.from_dict(final_dataset).T
paper_csv.head(2)

,cdate,content,ddate,details,forum,id,invitation,label,nonreaders,number,original,readers,referent,replyto,signatures,tcdate,tmdate,writers
2017_B1-Hhnslg,None,{'title': 'Prototypical Networks for Few-shot ...,None,{'replyCount': 9},B1-Hhnslg,B1-Hhnslg,ICLR.cc/2017/conference/-/submission,Reject,[],581,None,[everyone],None,None,[~Richard_Zemel1],1478376505434,1481060777491,[]
2017_B1-q5Pqxl,None,{'title': 'Machine Comprehension Using Match-L...,None,{'replyCount': 18},B1-q5Pqxl,B1-q5Pqxl,ICLR.cc/2017/conference/-/submission,Accept,[],417,None,[everyone],None,None,[~Shuohang_Wang1],1478290057546,1489370977828,[]


In [4]:
count_tldr = 0

for k in papers_data:
    if "TL;DR" not in papers_data[k]["content"]:
        count_tldr += 1

print(count_tldr)
print(len(paper_csv))

0
4897


In [5]:
# Extracting Title, Abstract, Authors, Keywords from data
paper_csv['title'] = [content['title'] for content in paper_csv['content']]
paper_csv['authors'] = [content['authors'] for content in paper_csv['content']]
paper_csv['abstract'] = [content['abstract'] for content in paper_csv['content']]
paper_csv['keywords'] = [content['keywords'] for content in paper_csv['content']]
paper_csv['tldr'] = [content["TL;DR"] for content in paper_csv['content']]

In [6]:
paper_csv.iloc[1]['content']

{'TL;DR': 'Using Match-LSTM and Answer Pointer to select a variable length answer from a paragraph',
 'abstract': 'Machine comprehension of text is an important problem in natural language processing. A recently released dataset, the Stanford Question Answering Dataset (SQuAD), offers a large number of real questions and their answers created by humans through crowdsourcing. SQuAD provides a challenging testbed for evaluating machine comprehension algorithms, partly because compared with previous datasets, in SQuAD the answers do not come from a small set of candidate answers and they have variable lengths. We propose an end-to-end neural architecture for the task. The architecture is based on match-LSTM, a model we proposed previously for textual entailment, and Pointer Net, a sequence-to-sequence model proposed by Vinyals et al. (2015) to constrain the output tokens to be from the input sequences. We propose two ways of using Pointer Net for our tasks. Our experiments show that both 

In [7]:
paper_csv.head(2)

,cdate,content,ddate,details,forum,id,invitation,label,nonreaders,number,...,replyto,signatures,tcdate,tmdate,writers,title,authors,abstract,keywords,tldr
2017_B1-Hhnslg,None,{'title': 'Prototypical Networks for Few-shot ...,None,{'replyCount': 9},B1-Hhnslg,B1-Hhnslg,ICLR.cc/2017/conference/-/submission,Reject,[],581,...,None,[~Richard_Zemel1],1478376505434,1481060777491,[],Prototypical Networks for Few-shot Learning,"[Jake Snell, Kevin Swersky, Richard Zemel]",A recent approach to few-shot classification c...,"[Deep learning, Transfer Learning]",We learn a metric space in which few-shot clas...
2017_B1-q5Pqxl,None,{'title': 'Machine Comprehension Using Match-L...,None,{'replyCount': 18},B1-q5Pqxl,B1-q5Pqxl,ICLR.cc/2017/conference/-/submission,Accept,[],417,...,None,[~Shuohang_Wang1],1478290057546,1489370977828,[],Machine Comprehension Using Match-LSTM and Ans...,"[Shuohang Wang, Jing Jiang]",Machine comprehension of text is an important ...,"[Natural language processing, Deep learning]",Using Match-LSTM and Answer Pointer to select ...


In [8]:
paper_csv['label'].unique()

array(['Reject', 'Accept'], dtype=object)

In [11]:
paper_csv.to_pickle('./features/all_data_features_17_20.pkl')